In [79]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator
from collections import Counter
import dateparser
import warnings

warnings.filterwarnings('ignore')

Попробую собрать первоначальные показатели с 2019 по март 2021 из ежедневных отчетов

In [80]:
dfs = pd.read_excel(r'\\tsclient\X\УМиР\2021\аналитика\Доли рынка\доля рынка.xlsx', sheet_name=None)

In [81]:
old_filials = pd.read_excel('Заубер Офисы.xlsx', sheet_name='Старое_ежедневное')

In [83]:
def to_date(x):
    if isinstance(x, str):
        return datetime.datetime.strptime(x, '%d.%m.%Y').date()
    else:
        return x

pattern = r'^.+\d{4}$'

profits, usd_incomes, usd_outcomes, euro_incomes, euro_outcomes, quants, cashboxes = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

indicators = ['Доход от валютно-обменных операций', 'Приход USD', 'Расход USD', 'Приход EUR', 'Расход EUR', 'Количество валютно-обменных операций', 'Остаток в кассе в нац. эквиваленте', 'Остаток в рублях']

for df in dfs:
    if re.match(pattern, df):
        temp_df = dfs[df].copy()
        temp_df = temp_df[[col for col in temp_df.columns if 'unnamed' not in str(col).lower()]]
        temp_df = temp_df.merge(old_filials, left_on='СВОД', right_on='Отделения', how='left')
        temp_df = temp_df[['Наим'] + [col for col in temp_df.columns if col != 'Наим']]
        temp_df.drop('Отделения', axis=1, inplace=True)
        temp_df = temp_df.dropna(subset=['СВОД'])
        temp_df = temp_df.reset_index(drop=True)
        try:
            indicators_rows_number = {element: temp_df['СВОД'][temp_df['СВОД'] == element].index[0] for element in indicators}
        except:
            print(df)
        new_values = list(zip(list(indicators_rows_number.values())[:-1], list(indicators_rows_number.values())[1:]))
        for key, value in indicators_rows_number.items():
            for i in new_values:
                if value in i:
                    indicators_rows_number[key] = i
        indicators_rows_number.pop('Остаток в рублях')
        profit, usd_income, usd_outcome, euro_income, euro_outcome, quant, cashbox = [temp_df.iloc[i[0]:i[1], :] for i in indicators_rows_number.values()]
        all_tables = [profit, usd_income, usd_outcome, euro_income, euro_outcome, quant, cashbox]
        for table in all_tables:
            table.reset_index(drop=True, inplace=True)
            new_name = table['СВОД'][0]
            table.rename(columns={'СВОД': new_name}, inplace=True)
            table.drop(0, inplace=True)
            table.drop(new_name, axis=1, inplace=True)
        def merge_df(x, y):
            if x.size == 0:
                x = x.append(y)
            else:
                x = x.merge(y, on='Наим', how='outer')
            return x    
        profits = merge_df(profits, profit)  
        usd_incomes = merge_df(usd_incomes, usd_income) 
        usd_outcomes = merge_df(usd_outcomes, usd_outcome) 
        euro_incomes = merge_df(euro_incomes, euro_income) 
        euro_outcomes = merge_df(euro_outcomes, euro_outcome) 
        quants  = merge_df(quants, quant) 
        cashboxes = merge_df(cashboxes, cashbox)
        
        
        profits.columns = ['Наим'] + [to_date(col) for col in profits.columns if col != 'Наим']
        usd_incomes.columns = ['Наим'] + [to_date(col) for col in usd_incomes.columns if col != 'Наим']
        usd_outcomes.columns = ['Наим'] + [to_date(col) for col in usd_outcomes.columns if col != 'Наим']
        euro_incomes.columns = ['Наим'] + [to_date(col) for col in euro_incomes.columns if col != 'Наим']
        euro_outcomes.columns = ['Наим'] + [to_date(col) for col in euro_outcomes.columns if col != 'Наим']
        quants.columns = ['Наим'] + [to_date(col) for col in quants.columns if col != 'Наим']
        cashboxes.columns = ['Наим'] + [to_date(col) for col in cashboxes.columns if col != 'Наим']
        
def stacked_df(df):
    df = pd.DataFrame(df.fillna(0).set_index('Наим').stack()).reset_index()
    df = df.rename(columns={'Наим': 'Филиал', 'level_1': 'Дата', 0: 'Показатель'})
    return df

profits = stacked_df(profits)
usd_incomes = stacked_df(usd_incomes)
usd_outcomes = stacked_df(usd_outcomes)
euro_incomes = stacked_df(euro_incomes)
euro_outcomes = stacked_df(euro_outcomes)
quants = stacked_df(quants)
cashboxes = stacked_df(cashboxes)

profits = profits.rename(columns={'Показатель': 'Доход'})
usd_incomes = usd_incomes.rename(columns={'Показатель': 'Приход USD'})
usd_outcomes = usd_outcomes.rename(columns={'Показатель': 'Расход USD'})
euro_incomes = euro_incomes.rename(columns={'Показатель': 'Приход EUR'})
euro_outcomes = euro_outcomes.rename(columns={'Показатель': 'Расход EUR'})
quants = quants.rename(columns={'Показатель': 'Кол-во операций'})
cashboxes = cashboxes.rename(columns={'Показатель': 'Остатов в кассе р.'})

all_df = profits.merge(usd_incomes, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')\
.merge(usd_outcomes, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')\
.merge(euro_incomes, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')\
.merge(euro_outcomes, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')\
.merge(quants, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')\
.merge(cashboxes, left_on=['Филиал', 'Дата'], right_on=['Филиал', 'Дата'], how='outer')

        

            
            


            

In [85]:
all_df = all_df.fillna(0)

In [86]:
all_df

Филиал       Дата  Доход  Приход USD  Расход USD  Приход EUR  \
0         ГО 2020-01-01    0.0         0.0         0.0         0.0   
1         ГО 2020-01-02    0.0         0.0         0.0         0.0   
2         ГО 2020-01-03    0.0         0.0         0.0         0.0   
3         ГО 2020-01-04    0.0         0.0         0.0         0.0   
4         ГО 2020-01-05    0.0         0.0         0.0         0.0   
...      ...        ...    ...         ...         ...         ...   
26855      0 2019-12-27    0.0         0.0         0.0         0.0   
26856      0 2019-12-28    0.0         0.0         0.0         0.0   
26857      0 2019-12-29    0.0         0.0         0.0         0.0   
26858      0 2019-12-30    0.0         0.0         0.0         0.0   
26859      0 2019-12-31    0.0         0.0         0.0         0.0   

       Расход EUR  Кол-во операций  Остатов в кассе р.  
0             0.0              0.0        1.525036e+08  
1             0.0              0.0        1.525335e+08  
2             0.0              0.0        1.525335e+08  
3             0.0              0.0        1.525335e+08  
4             0.0              0.0        1.525335e+08  
...           ...              ...                 ...  
26855         0.0              0.0        0.000000e+00  
26856         0.0              0.0        0.000000e+00  
26857         0.0              0.0        0.000000e+00  
26858         0.0              0.0        0.000000e+00  
26859         0.0              0.0        0.000000e+00  

[26860 rows x 9 columns]

In [94]:
all_df['Дата'] = all_df['Дата'].apply(lambda x: x.date())

In [110]:
all_df['Филиал'].value_counts()

all_df.drop(all_df[all_df['Филиал'].isin([0, 'Новые Черемушки, Москва - ОКВКУ №11'])].index, inplace=True)

In [111]:
all_df['Филиал'].value_counts()

Тульская - ОКВКУ 24         790
Рижская-ОКВКУ 5             790
Парк Победы-ОКВКУ 7         790
Испытателей-ОКВКУ 3         790
Филиал в г. Черкесск        790
Новые Черемушки-ОКВКУ 11    790
Московский-ДО 2             790
Войковская - ОКВКУ 20       790
Белорусская-ОКВКУ 10        790
Озерки-ОКВКУ 12             790
Пресня-ДО 4                 790
Семеновская-ОКВКУ 14        790
Звездная-ДО 6               790
Бибирево-ДО 5               790
Братиславская-ОКВКУ 16      790
Павелецкая-ОКВКУ 8          790
Кузьминки - ОКВКУ 21        790
Жулебино-ДО 3               790
Международная-ОКВКУ 4       790
ГО                          790
Спортивная-ОКВКУ 15         790
Люблино - ОКВКУ 18          790
Лиговский-ОКВКУ 2           790
Чертаново - ОКВКУ 23        790
Крылатское-ОКВКУ 9          790
Марата-ДО 1                 790
Наличная-ОКВКУ 13           790
Заубер Сити - ОКВКУ 19      790
Полежаевская - ОКВКУ 22     790
Теплый стан-ОКВКУ 17        790
Комендантский-ОКВКУ 1       790
Красные 

## Вставим в mysql базу bank_statistics

In [87]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import datetime
import re

In [112]:
all_df_sql = [tuple(row) for index, row in all_df.iterrows()]

In [114]:
# вставим значения из df


# проверим созданную таблицу

try:
    with connect(
        host="localhost",
        user='root',
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = """INSERT INTO aggregate_indicators
        (Филиал, 
        Дата, 
        Доход,
       Приход_USD, 
       Расход_USD, 
       Приход_EUR,
       Расход_EUR,
       Кол_во_операций, 
       Остаток_в_кассе_кубли)
       VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        with connection.cursor() as cursor:
            cursor.executemany(query, all_df_sql)
            connection.commit()
    
except Error as e:
    print(e)

Пароль: ········


In [106]:
all_df[all_df['Филиал'] =='Новые Черемушки-ОКВКУ 11']

Филиал        Дата          Доход  Приход USD  \
12640  Новые Черемушки-ОКВКУ 11  2020-01-01       0.000000         0.0   
12641  Новые Черемушки-ОКВКУ 11  2020-01-02       0.000000         0.0   
12642  Новые Черемушки-ОКВКУ 11  2020-01-03       0.000000         0.0   
12643  Новые Черемушки-ОКВКУ 11  2020-01-04       0.000000         0.0   
12644  Новые Черемушки-ОКВКУ 11  2020-01-05       0.000000         0.0   
...                         ...         ...            ...         ...   
13425  Новые Черемушки-ОКВКУ 11  2019-12-27  209030.116571    135225.0   
13426  Новые Черемушки-ОКВКУ 11  2019-12-28  280325.882963    186109.0   
13427  Новые Черемушки-ОКВКУ 11  2019-12-29   78223.964447     77963.0   
13428  Новые Черемушки-ОКВКУ 11  2019-12-30  249122.877820     92450.0   
13429  Новые Черемушки-ОКВКУ 11  2019-12-31       0.000000         0.0   

       Расход USD  Приход EUR  Расход EUR  Кол-во операций  Остатов в кассе р.  
12640         0.0        0.00        0.00              0.0           118474.56  
12641         0.0        0.00        0.00              0.0           118474.56  
12642         0.0        0.00        0.00              0.0           118474.56  
12643         0.0        0.00        0.00              0.0           118474.56  
12644         0.0        0.00        0.00              0.0           118474.56  
...           ...         ...         ...              ...                 ...  
13425    401124.0   133067.84   219387.84            135.0         52480787.56  
13426    316123.0    87904.12    88484.12            133.0         85040831.60  
13427     86552.0    23445.40    41095.40            100.0         85687207.99  
13428    520104.0   110230.00   191370.00             68.0         85787457.65  
13429         0.0        0.00        0.00              0.0                0.00  

[790 rows x 9 columns]